In [1]:
import os
import re
import datetime as dt
import pandas as pd

In [2]:
desc ='01/03/2021 Morgan M. Mack eCheck receipt C712-17B0 Rental Income - January 2021 1,350.00 1,220.45'
desc2='01/11/2021 Yinong Jiang Check 14096 Owner Distribution - Owner payment for 01/2021 1,085.45 0.00'
desc3='Toho Water Authority Receipt Water'

m = re.search('(.*) ((e?Check)(receipt)?) ([A-Za-z0-9-]+) (.*)',desc3) 

#print(m.group(2))



    

In [3]:
def isNumber(txt):
    try:
        float(txt.replace(',',''))
        return True
    except:
        return False

def str2num(txt):
    return float(txt.replace(',',''))

def str2date(txt):
    return dt.datetime.strptime(txt,'%m/%d/%Y')
    
def isDate(txt):
    try:
        dt.datetime.strptime(txt,'%m/%d/%Y')
        return True
    except:
        return False
        

In [4]:
PAYEES=['Yinong Jiang','Hathaway Home Services Results Realty']

In [5]:
class Record:

    #---------------------------------------------------
    # Main logic of Income expesne
    #---------------------------------------------------
    
    def setIncomeExpense(self,num):
        if( 'Application Fee Income - Refund' in self.Description):
            self.Expense = num       
        elif( self.Type == 'Check'):
            self.Expense = num                   
        elif( ('Income' in self.Description) | ('Prepaid' in self.Description) ):
            self.Income = num
        elif( re.search('security deposit',self.Description,re.IGNORECASE) ):
            self.Income = num
        elif( re.search('receipt',self.Type,re.IGNORECASE)) :
            self.Income = num
        elif( re.search('refund',self.Description,re.IGNORECASE)) :
            self.Income = num
        elif( re.search('final bill close',self.Description,re.IGNORECASE)) :
            self.Income = num
        elif( re.search('Transfer from',self.Description,re.IGNORECASE)) :
            self.Income = num            
        elif( re.search('receipt water',self.Description,re.IGNORECASE)) :
            self.Income = num 
        else:
            self.Expense = num

    #----------------------------------------------------
    # Main logic for payee/type/reference/description
    #----------------------------------------------------

    def setFields(self,desc):
        
        m = None
        if( 'Check' in desc ):
            m = re.search('(.*) ((e?Check)?( ?receipt)?) ([A-Za-z0-9-]+) (.*)',desc)
        
        if( m != None ):
            self.Payee = m.group(1)
            self.Type = m.group(2)
            self.Reference = m.group(5)
            self.Description = m.group(6)
        else:
            self.Description = desc

    # Set Tax Type
    def setTaxType(self):
        if( re.search('Owner Distribution',self.Description,re.IGNORECASE)):
            self.TaxType = 'Owner'
        elif( re.search('Legal',self.Description)):
            self.TaxType = 'Legal'
        elif( re.search('Rental Income',self.Description,re.IGNORECASE)):
            self.TaxType = 'Rental Income'
        elif( re.search('Prepaid Rent',self.Description,re.IGNORECASE)):
            self.TaxType = 'Rental Income'            
        elif( re.search('Rental Income',self.Description,re.IGNORECASE)):
            self.TaxType = 'Rental Income'
        elif( re.search('Management Fees',self.Description,re.IGNORECASE)):
            self.TaxType = 'Management Fee'
        elif( re.search('Transfer',self.Description,re.IGNORECASE)):
            self.TaxType = 'Transfer' 
        # ---------------------------
        # utility
        elif( re.search('water',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Utility'
        elif( re.search('electric',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Utility'
        elif( re.search('electricity',self.Description,re.IGNORECASE)):
            self.TaxType = 'Utility'            
        elif( re.search('water',self.Description,re.IGNORECASE)):
            self.TaxType = 'Utility'            
            
        #----------------------------
        # repaires
        elif( re.search('Repaire',self.Description,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Elizabeth Wallas',self.Description,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Elizabeth Wallas',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Crain\'s Mechanical',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('ACH Payment',self.Description,re.IGNORECASE)):
            self.TaxType = 'Repair'            
        elif( re.search('RV Gas Appliance',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Carpet',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
        elif( re.search('Bugs B Gone',self.Payee,re.IGNORECASE)):
            self.TaxType = 'Repair'
            
        #----------------------------
        # Admin fees
        elif( re.search('Application',self.Description,re.IGNORECASE)):
            self.TaxType = 'Admin'            
        elif( re.search('Admin fee',self.Description,re.IGNORECASE)):
            self.TaxType = 'Admin'            
        elif( re.search('Late fee',self.Description,re.IGNORECASE)):
            self.TaxType = 'Admin'            
            
            
    def __init__(self,Date):
        self.Date = Date
        self.Payee =' '
        self.Type =' '
        self.TaxType = ' '
        self.Reference =' '
        self.Description =' '
        self.Income=0
        self.Expense=0
        self.Balance=0
        
    def __str__(self):
        return f'{self.Date}|{self.Payee}|{self.Type}|{self.Reference}|{self.Description}|{self.Income}|{self.Expense}|{self.Balance}'

    def tostr(self):
        return f'{self.Date}|{self.Payee}|{self.Type}|{self.TaxType}|{self.Reference}|{self.Description}|{self.Income}|{self.Expense}|{self.Balance}'
                
#-------------------------------------------------------------
# Test code
r = Record(dt.datetime.strptime('1/1/2021','%m/%d/%Y'))

In [11]:
def readBeginingBalance(fin,fout):
    # wait for 'Beginning'
    line = fin.readline()
    while( not line.startswith('Beginning')):
        line = fin.readline()
    m = re.search('(Beginning.*)(\d+/\d+/\d+) (-?\d+.\d{2})',line)

    r = Record(dt.datetime.strptime(m.group(2),'%m/%d/%Y'))
    r.Balance = m.group(3)
    
    fout.write(r.tostr() + '\n')
    #print(r.tostr())


In [14]:
def readOneRecord(fin,fout):
    line = ''
    p = re.compile('.* -?[0-9,]+[.]\d{2}$')
    while(True):
        readline = fin.readline()
        if( not readline ):
            return False
        line = line + readline.replace('\n','')
        if( p.match(line) != None): 
            break
        line = line + ' '
    
    # now got full line
    line.replace('\n','')
    #print(line)
    m = re.search('^(\d+/\d+/\d+) (.*) (-?[0-9,]+.\d{2}) (-?[0-9,]+.\d{2})',line)
    #print(m)
    #print(line)
    if( m ):
        r = Record(str2date(m.group(1)))
        r.setFields(m.group(2))
        r.setIncomeExpense(str2num(m.group(3)))
        r.Balance = str2num(m.group(4))
    
        r.setTaxType()
        
        fout.write(r.tostr() + '\n')

    return True



In [21]:
def createData(year,house):

    DIR=f'E:\\Documents\\{year} tax'

    file_name_in = f'{DIR}\\{house}_in.txt'
    file_name_out= f'{DIR}\\{house}_out.txt'
    
    fin = open(file_name_in,'r')
    fout = open(file_name_out,'w')


    start = True

    fout.write('Date|Payee|Type|TaxType|Reference|Descrition|Income|Expense|Balance\n')
    readBeginingBalance(fin,fout)
    while(readOneRecord(fin,fout)):
        continue;

    fin.close()
    fout.close()
    return file_name_out


In [22]:
print(f'Now:{dt.datetime.now()}')

file_name = createData('2021','5311')

df = pd.read_csv(file_name,delimiter='|')

df

Now:2022-03-14 10:34:11.648705


,Date,Payee,Type,TaxType,Reference,Descrition,Income,Expense,Balance
0,2021-01-01 00:00:00,,,,,,0.0,0.0,"1,17"
1,2021-01-02 00:00:00,Anibal E. Sanchez eCheck,receipt,Rental Income,9C7E-6A90,Rental Income - January 2021,1300.0,0.0,2475.0
2,2021-01-11 00:00:00,Berkshire Hathaway HomeServices Results Realty,Check,Management Fee,14066,Management Fees - Management Fees for 01/2021,0.0,130.0,2345.0
3,2021-01-11 00:00:00,Yinong Jiang,Check,Owner,14096,Owner Distribution - Owner payment for 01/2021,0.0,1170.0,1175.0
4,2021-02-02 00:00:00,Anibal E. Sanchez eCheck,receipt,Rental Income,B94E-6340,Rental Income - February 2021,1300.0,0.0,2475.0
...,...,...,...,...,...,...,...,...,...
82,2021-11-10 00:00:00,Yinong Jiang,Check,Owner,15261,Owner Distribution - Owner payment for 11/2021,0.0,1215.0,1175.0
83,2021-11-24 00:00:00,Rodriguez Nelson eCheck,receipt,Rental Income,7038-9A10,Prepaid Rent - Online Payment,1350.0,0.0,2525.0
84,2021-12-10 00:00:00,Berkshire,Check,Management Fee,15361,Management Fees - Management Fees for 12/2021,0.0,135.0,2390.0
85,2021-12-10 00:00:00,Yinong Jiang,Check,Owner,15391,Owner Distribution - Owner payment for 12/2021,0.0,1215.0,1175.0


In [28]:
#print(f'Now:{dt.datetime.now()}')

print('Created following data files:\n')

print(createData('2021','5311'))
print(createData('2021','5510'))
print(createData('2021','5212'))

Created following data files:

E:\Documents\2021 tax\5311_out.txt
E:\Documents\2021 tax\5510_out.txt
E:\Documents\2021 tax\5212_out.txt
